<a href="https://colab.research.google.com/github/maririv/03MAIR-Algortimos-de-Optimizacion/blob/main/Algoritmo_INES_MARI_RIVERO_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Actividad guiada 3

Inés Marí Rivero

https://github.com/maririv

#### Instalar librerias necesarias

In [ ]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.8 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


#### Carga de los datos del problema

In [ ]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random                     #Para generar valores aleatorios

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos



gzip: swiss42.tsp already exists; do you wish to overwrite (y or n)? y


In [ ]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Probamos algunas funciones del objeto problem

#Distancia entre nodos
problem.get_weight(0, 1)

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html

#dir(problem)

NameError: ignored

#### Funciones basicas


In [ ]:
#Funciones basicas
###############################################################################

#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]] #creamos solucion con el nodo 0
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem) #distancia total + distancia al punto cero

In [ ]:
solucion = [Nodos[0]]
for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))] #elige un elemento de los que le quedan disponibles
print(solucion)

[0, 41, 5, 37, 33, 3, 17, 21, 31, 7, 23, 36, 20, 4, 30, 2, 18, 6, 9, 38, 11, 12, 15, 13, 19, 10, 40, 35, 28, 27, 16, 8, 24, 22, 14, 34, 32, 1, 39, 29, 26, 25]


#### BUSQUEDA ALEATORIA

In [ ]:
###############################################################################
# BUSQUEDA ALEATORIA
###############################################################################

def busqueda_aleatoria(problem, N):
  #N es el numero de iteraciones
  Nodos = list(problem.get_nodes())

  mejor_solucion = []
  #mejor_distancia = 10e100                         #Inicializamos con un valor alto
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto porque vamos a quedarnos con la menor distancia

  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)

    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia


  print("Mejor solución:" , mejor_solucion)
  print("Distancia     :" , mejor_distancia)
  return mejor_solucion


#Busqueda aleatoria con 5000 iteraciones
solucion = busqueda_aleatoria(problem, 100000)

Mejor solución: [0, 38, 24, 22, 13, 14, 33, 34, 18, 6, 35, 3, 10, 27, 21, 40, 39, 9, 17, 7, 20, 5, 25, 26, 32, 28, 4, 30, 2, 19, 29, 41, 8, 12, 11, 23, 15, 16, 36, 37, 31, 1]
Distancia     : 3574


#### BUSQUEDA LOCAL: intentamos llegar a un mínimo desde la solución inicial generada

In [ ]:
###############################################################################
# BUSQUEDA LOCAL
###############################################################################
#solucion = crear_solucion(Nodos)
solucion = busqueda_aleatoria(problem, 100000)
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):

      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:] # swap
      #solución hasta i + j es el que voy a intercambiar + desde i+1 hasta j + elementoo i + el resto

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion


#solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))


nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))


Mejor solución: [0, 22, 39, 31, 7, 33, 27, 29, 9, 38, 6, 1, 3, 16, 19, 34, 20, 18, 36, 37, 35, 17, 14, 15, 28, 25, 41, 21, 23, 12, 32, 13, 10, 30, 24, 40, 11, 8, 4, 5, 26, 2]
Distancia     : 3490
Distancia Solucion Incial: 3490
Distancia Mejor Solucion Local: 3168


In [ ]:
#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
# ahora cada vez que encontremos una mejor solución la usamos como solución de partida.
#Voy a moverme entre los vecinos en la mejor dirección hasta que ya no encuentre una mejor solución
def busqueda_local(problem):
  mejor_solucion = []

  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)
  print(mejor_distancia)
  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)

    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      #mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia
      mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:

      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion)
      print("Distancia     :" , mejor_distancia)
      return mejor_solucion

    solucion_referencia = vecina


sol = busqueda_local(problem )

5244
En la iteracion  47 , la mejor solución encontrada es: [0, 7, 17, 31, 36, 35, 20, 33, 34, 32, 28, 2, 27, 1, 37, 15, 16, 14, 26, 10, 8, 9, 29, 30, 38, 22, 39, 21, 24, 40, 23, 41, 25, 11, 12, 18, 13, 19, 5, 6, 4, 3]
Distancia     : 1492


In [ ]:
# Se puede mejorar con multiarranque
# se puede dar algo de voracidad a la solución inicial -- GRASP (parte de greedy pero le mete aleatoriedad)
# GRASP puede ser completamente voraz o completamente aleatorio(si la lista de candidatos restringida se abre a todas las opciones)
# Se puede "jugar" calibrando la lista de candidatos
# una opcion  la probabilidad sea proporcional al inverso multiplicativo de la distancia
# otra opcion: en cada iteraccion definir el tamaño de la lista. Evaluo una solucion, cojo la mejor y la peor distancia
# y decido que entren todos los que sean menores a (1-alfa)*mejor +alfa*peor (con alfa entre 0 y 1)
# si alfa = 1: busqueda aleatoria
# si alfa = 0: completamente voraz
# ir calibrando alfa
# también se puede ir modificando alfa (aumentando la aleatoriedad como va pasando el tiempo, por ejemplo)

In [ ]:
#para mejorar la nota: búsqueda en entornos variables (método exhaustivo -VNS- prueba diversos operadores (swap, insertion
#### ) ) -> utilizamos el swap hasta que no mejora, ahí pasamos a la inserción
#mejora: que en vez de parar pasemos a otra fase por ejemplo volver al inicio
# lo mejor es darle una condicion de tiempo, no de numero de iteraciones: por ejemplo el numero de nodos
#probar, en vez del intercambio, la reinserción

#propuesta mejora nota:
- probar VNS
- multiarranque
- GRASP